# `clip_retrieval.clip_client`

This python module allows you to query a backend remote via its exposed REST api.

## Install

In [ ]:
%pip install clip-retrieval img2dataset

## Setup

In [ ]:
from IPython.display import Image, display
from clip_retrieval.clip_client import ClipClient, Modality

IMAGE_BASE_URL = "https://github.com/rom1504/clip-retrieval/raw/main/tests/test_clip_inference/test_images/"

def log_result(result):
    id, caption, url, similarity = result["id"], result["caption"], result["url"], result["similarity"]
    print(f"id: {id}")
    print(f"caption: {caption}")
    print(f"url: {url}")
    print(f"similarity: {similarity}")
    display(Image(url=url, unconfined=True))

client = ClipClient(
    url="https://knn5.laion.ai/knn-service",
    aesthetic_score=9,
    aesthetic_weight=0.5,
    modality=Modality.IMAGE,
    num_images=1,
    num_result_ids=3000,
)


## Query by text

In [14]:
cat_results = client.query(text="an image of a cat")
log_result(cat_results[0])

id: 518836491
caption: orange cat with supicious look stock photo
url: https://media.istockphoto.com/photos/orange-cat-with-supicious-look-picture-id907595140?k=6&amp;m=907595140&amp;s=612x612&amp;w=0&amp;h=4CTvSxNvv4sxSCPxViryha4kAjuxDbrXM5vy4VPOuzk=
similarity: 0.5591725707054138


## Query by image

In [15]:
beach_results = client.query(image="https://github.com/rom1504/clip-retrieval/raw/main/tests/test_clip_inference/test_images/321_421.jpg")
log_result(beach_results[0])

id: 574870177
caption: Palm trees in Orlando, Florida
url: https://www.polefitfreedom.com/wp-content/uploads/2018/03/Orlando.jpg
similarity: 0.9619366526603699


## Download and format a dataset from the results of a query

If you have some images of your own, you can query each one and collect the results into a custom dataset (a small subset of LAION-5B)

In [ ]:
# Create urls from known images in repo
import json
from tqdm import tqdm
test_images = [f"{IMAGE_BASE_URL}{image}" for image in ["123_456.jpg", "208_495.jpg", "321_421.jpg", "389_535.jpg", "416_264.jpg", "456_123.jpg", "524_316.jpg"]]

# Re-initialize client with higher num_images
client = ClipClient(url="https://knn5.laion.ai/knn-service", num_images=40)

# Run one query per image
combined_results = []
for image in tqdm(test_images):
    combined_results.extend(client.query(image=image))

# Save results to json file
with open("search-results.json", "w") as f:
    json.dump(combined_results, f)

In [ ]:
!img2dataset "search-results.json" --input_format="json" --output_folder="laion-enhanced-images" --resize_mode="no" --output_format="webdataset"

In [ ]:
import os
print(f"Done! Download/copy the contents of {os.getcwd()}/laion-enhanced-images/")
!realpath laion-enhanced-images/*.tar